In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
plaid_ids = pd.read_csv('/Users/shashankgupta/Documents/code/git_project/plaid_credit/Data/base_plaid_coverage.csv')
plaid_ids.shape

(710, 4)

In [3]:
plaid_ids.head()

,LENDING_BUSINESS_ID,BUSINESS_ID,Ever15plus,Source
0,b88bad24-8711-444a-98e1-52047e87f6fc,fdbeb997-87d8-411e-a068-9a2297324910,0,GA
1,987da0d5-f838-45a9-8bb6-d19a64e8888c,215ca059-aac1-4c09-a6db-9e2a143ce7c5,0,GA
2,166d62fb-70ed-4214-8a75-1223dfbbd06e,252c9a3f-338f-43fc-893f-87e4c8184e8c,0,GA
3,8105f405-a33a-434e-9f0d-0c0fed3b0889,02d133cb-e8eb-449d-9b7d-d0501058bc61,0,GA
4,13fe3447-98d9-4996-92e5-fffa8e63429b,e98ced58-b53b-44c0-8cfa-2dd9b43f95be,0,GA


In [4]:
plaid_ids = plaid_ids.rename(columns={'LENDING_BUSINESS_ID': 'lending_business_id', 'BUSINESS_ID': 'business_id'})

In [5]:
bids_tuple = tuple(plaid_ids['business_id'])
lid_tuple = tuple(plaid_ids['lending_business_id'])

In [6]:
querySno = SQLQuery('snowflake')
biz_info = querySno(r""" 
select id as business_id, JSON_EXTRACT_PATH_TEXT(meta, 'business_questions.industry_category_name') as industry_category_name, 
JSON_EXTRACT_PATH_TEXT(meta, 'business_questions.industry_name') as industry_name, 
JSON_EXTRACT_PATH_TEXT(meta, 'business_questions.number_of_employees') as number_of_employees
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."BUSINESSES"
where id in {}
""".format(bids_tuple))

In [7]:
biz_info.shape

(709, 4)

In [8]:
querySno = SQLQuery('snowflake')
rev30_info = querySno(r""" 
select lending_business_id,
JSON_EXTRACT_PATH_TEXT(decision_variables, 'context.rev_30') as rev_30,
JSON_EXTRACT_PATH_TEXT(decision_variables, 'context.novo_risk_model_score') as novo_risk_model_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_APPLICATION_SUBMISSION_VARIABLES
where lending_business_id in {}
""".format(lid_tuple))

In [9]:
rev30_info.shape

(707, 3)

In [10]:
querySno = SQLQuery('snowflake')
rs2_df = querySno(r""" 
select lending_business_id, 
JSON_EXTRACT_PATH_TEXT(predict_meta, 'default_bin') as rs2_bin,
JSON_EXTRACT_PATH_TEXT(predict_meta, 'novo_risk_model_score') as rs2_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS
where lending_business_id in {}
""".format(lid_tuple))

In [11]:
rs2_df.shape

(707, 3)

In [12]:
t1 = plaid_ids.merge(biz_info, how = 'left', left_on = 'business_id', right_on = 'business_id')
t2 = t1.merge(rev30_info, how='left', left_on = 'lending_business_id', right_on = 'lending_business_id')
df_raw = t2.merge(rs2_df, how='left', left_on = 'lending_business_id', right_on = 'lending_business_id')

In [13]:
df_raw.sample(5)

,lending_business_id,business_id,Ever15plus,Source,industry_category_name,industry_name,number_of_employees,rev_30,novo_risk_model_score,rs2_bin,rs2_score
695,55d04b92-7adb-4938-a8ce-f4241d5c59c0,5248448f-dc61-4699-b87d-20c1203ed386,1,FICO,Retail Trade,All Other Miscellaneous Store Retailers (excep...,1,2673.0332,573,3,573
144,ceb0653a-4981-4a72-962d-f71adac3f809,98dd3072-f3ed-4b6c-8896-e1f516273027,0,GA,Information,Motion Picture and Video Production,1,3265.0207,626,4,626
577,2cde824c-01df-4076-a128-d0404668f766,9b66659c-529c-4e91-8705-a61a7bddfd13,1,FICO,Retail Trade,Electronic Shopping and Mail-Order Houses,1,None,None,1,358
15,1888c110-974e-4fa0-8d85-6848857700de,847daccf-5011-44c9-a71b-83267581f92a,1,GA,Administrative and Support and Waste Managemen...,Other Waste Collection,1,None,None,3,551
346,cd81e279-ddf1-4452-860a-7cf093444f11,a4f15690-8691-4705-95da-36517e139eb9,0,GA,Accommodation and Food Services,Drinking Places (Alcoholic Beverages),2,16184.7832,896,5,896


In [14]:
df_raw.isnull().sum()

lending_business_id        0
business_id                0
Ever15plus                 0
Source                     0
industry_category_name     0
industry_name              0
number_of_employees       34
rev_30                    87
novo_risk_model_score     87
rs2_bin                   30
rs2_score                 30
dtype: int64

In [15]:
df_raw.groupby('rs2_bin', as_index=False)['lending_business_id'].agg('count')

,rs2_bin,lending_business_id
0,1,18
1,2,31
2,3,163
3,4,175
4,5,293


In [16]:
import plotly.express as px

fig = px.bar(df_raw.groupby(['rs2_bin'], as_index=False)['lending_business_id'].count(), x='rs2_bin', y='lending_business_id', 
                text_auto=True)
fig.show()

In [17]:
querySno = SQLQuery('snowflake')
credit_df = querySno(r""" 
select business_id, transaction_id,date, amount, description, merchant_name, type, categories, 
datediff(day,date, current_timestamp) as days_from_current   
from "METABASE_DB"."TRANSACTION"."PLAID_TRANSACTIONS"
where type = 'credit'
and datediff(day,date, current_timestamp) <= 180
and business_id in {}
""".format(bids_tuple))

In [18]:
credit_df.shape

(65039, 9)

In [90]:
querySno = SQLQuery('snowflake')
credit_df = querySno(r""" 
with target_businesses as (
SELECT DISTINCT a.id AS business_id, APPLICATION_ID,
lower(regexp_replace(a.company_name, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS company_name,
CASE WHEN dba IS NOT NULL THEN lower(regexp_replace(dba, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) ELSE NULL end AS dba,
case when a.TYPE = 'sole_proprietorship' then 'sol_prop' else 'non sol_prop' end AS business_type
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC.BUSINESSES a
join prod_db.data.businesses b
on a.id = b.business_id
where a.id IN (
    select business_id
    from prod_db.adhoc.external_model_training_base)

)

,logic1 as (
-- account name match
SELECT distinct t1.id as plaid_item_id, t1.business_id, json_extract_path_text(json_extract_path_text(t2.value,'account'),'name') as ext_account_name,
lower(regexp_replace(json_extract_path_text(json_extract_path_text(t2.value,'account'),'name'), '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS external_acc_company_name ,
json_extract_path_text(json_extract_path_text(t2.value,'ach'),'account') AS account_id
FROM FIVETRAN_DB.plaid_data_public.plaid_items t1, LATERAL FLATTEN(input => "META") t2
WHERE status = 'active'
and account_id is not null
and account_id != ''
union
SELECT distinct t1.id as plaid_item_id, t1.business_id, t2.value as ext_account_name,
lower(regexp_replace(t2.value, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS external_acc_company_name, 
--'' as account_id,
json_extract_path_text(json_extract_path_text(t3.value,'ach'),'account') AS account_id
FROM FIVETRAN_DB.plaid_data_public.plaid_items t1, LATERAL FLATTEN(input => "IDENTITY":names) t2
, LATERAL FLATTEN(input => "META") t3
WHERE status = 'active'
and account_id is not null
and account_id != ''
order by account_id
),
mapped_accts as (
select distinct a.*, b.plaid_item_id, b.ext_account_name, b.external_acc_company_name, b.account_id, e.owned_by,
case when ((company_name = external_acc_company_name) OR 
(SOUNDEX(company_name) = SOUNDEX(external_acc_company_name)) OR
(dba IS NOT NULL AND dba = external_acc_company_name) OR
(dba IS NOT NULL AND SOUNDEX(dba)=SOUNDEX(external_acc_company_name))) then 1 else 0 end as name_match_flag
from target_businesses a
left join logic1 b 
on a.business_id = b.business_id
left join metabase_db.external.external_accounts e
on a.business_id = e.business_id
and b.account_id = e.account_number
where account_id is not null
and account_id !=''
), --6.78%
logic2 as (
-- email id match
select distinct a.business_id, a.company_name, email, email_domain, website
from target_businesses a
left join PROD_DB.DATA.APPLICATIONS b
on a.application_id = b.application_id
),
plaid_email as (
select distinct business_id, t1.id as plaid_item_id, json_extract_path_text(t2.value,'data') as email,
json_extract_path_text(json_extract_path_text(t3.value,'ach'),'account') AS account_id
from fivetran_db.plaid_data_public.plaid_items t1, LATERAL FLATTEN(input => "IDENTITY":emails) t2
, LATERAL FLATTEN(input => "META") t3
where status = 'active'
),
match_email as (
select b.*, plaid_item_id, lower(t1.email) as plaid_email, account_id,
--split_part(b.email,'@',1),split_part(b.email,'@',2),
case when (email_domain not in ('gmail.com','yahoo.com','outlook.com','hotmail.com','icloud.com','protonmail.com',
'me.com','live.com','mail.com','comcast.net','att.net','msn.com','pm.me','mac.com') and 
          soundex(b.email_domain) = soundex(split_part(plaid_email,'@',2)) )
          or b.email = plaid_email
          or b.email_domain = split_part(plaid_email,'@',1)
          then 1 else 0 end as email_match_flag
from logic2 b
left join plaid_email t1
on t1.business_id = b.business_id
order by business_id
), 
logic4 as (
--website match
select distinct b.business_id, p.plaid_item_id, p.account_id, a.external_acc_company_name, lower(split_part(replace(b.website,'www.',''),'.',1)) as cleaned_website, 
case when (b.website is not null) and ((soundex(external_acc_company_name) = soundex(cleaned_website)) 
          or (external_acc_company_name = cleaned_website)
          or (cleaned_website = split_part(split_part(p.email,'@',2),'.',1))
          or (cleaned_website = split_part(p.email,'@',1))
          or (soundex(cleaned_website) = soundex(split_part(split_part(p.email,'@',2),'.',1))) 
          or (soundex(cleaned_website) = soundex(split_part(p.email,'@',1)))) 
          then 1 else 0 end as website_match
from logic2 b
left join plaid_email p
on b.business_id = p.business_id
left join logic1 a
on a.business_id = b.business_id
and p.account_id = a.account_id
),
final as (
select distinct a.business_id, a.company_name, a.dba, a.external_acc_company_name,
case when a.plaid_item_id is null and d.plaid_item_id is null then b.plaid_item_id
     when a.plaid_item_id is null and b.plaid_item_id is null then d.plaid_item_id
     else a.plaid_item_id end as plaid_item_id, 
case when a.account_id is null and d.account_id is null then b.account_id
     when a.account_id is null and b.account_id is null then d.account_id
     else a.account_id end as account_id, 
a.owned_by,
b.email, b.plaid_email,cleaned_website,
max(a.name_match_flag) as name_match_flag,
max(b.email_match_flag) as email_match_flag,
--max(c.comp_zip_match) as zip_match_flag,
max(website_match) as website_match_flag
from target_businesses s
left join mapped_accts a
on s.business_id = a.business_id
left join match_email b
on a.business_id = b.business_id
and a.account_id = b.account_id
left join logic4 d
on d.business_id = s.business_id
and a.account_id = d.account_id
group by 1,2,3,4,5,6,7,8,9,10
),
external_accts as (
select *,
case when name_match_flag=1 or email_match_flag=1 or website_match_flag=1
then 1 else 0 end as final_flag
from final
),
ext_accounts as (
select distinct a.business_id, --a.plaid_item_id, 
a.account_id,
d.id as EXTERNAL_ACCOUNT_ID
from external_accts a
left join METABASE_DB.EXTERNAL.EXTERNAL_ACCOUNTS d
on a.account_id = d.account_number
and a.account_id is not null
and a.account_id!=''
where a.final_flag=1
and (a.owned_by is null or a.owned_by='' or a.owned_by='business')
and EXTERNAL_ACCOUNT_ID is not null
order by business_id
)

select *, datediff(day,b.date, date('2023-06-20')) as days_from_current from ext_accounts a 
left join "METABASE_DB"."TRANSACTION"."PLAID_TRANSACTIONS" b 
on a.business_id = b.business_id and a.EXTERNAL_ACCOUNT_ID = b.external_account_id
where b.external_account_id is not null and b.type = 'credit' and datediff(day,b.date, date('2023-06-20')) <= 180

""")

In [91]:
credit_df = pd.read_csv('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/plaid_credit_transactions.csv')

In [115]:
credit_df.iloc[:,12:24].to_clipboard()

In [117]:
to_copy = credit_df.drop('META',axis=1)
to_copy.to_clipboard()

In [92]:
credit_df.shape

(17020, 24)

In [95]:
df = credit_df.merge(df_raw,how='left',left_on='BUSINESS_ID',right_on='business_id')

In [96]:
df.shape

(17040, 35)

In [98]:
credit_df[~credit_df['MERCHANT_NAME'].isnull()].shape

(1777, 24)

In [100]:
credit_df['MERCHANT_NAME'].value_counts()

BetMGM         205
eBay           122
Amazon         106
Sezzle          66
Walmart         65
              ... 
Cleo             1
Refu             1
RaceWay          1
Whataburger      1
Stop             1
Name: MERCHANT_NAME, Length: 235, dtype: int64

In [103]:
credit_df['CATEGORIES'].value_counts().to_clipboard()

In [24]:
credit_df[credit_df['CATEGORIES']=='Transfer,Credit']['MERCHANT'].value_counts()

Earnin             144
eBay                92
SoFi                56
AfterPay            39
Dave                38
                  ... 
ZKW                  1
Rudyjude             1
Gui Fang             1
Wisestamp            1
Tx P Realty LLC      1
Name: merchant_name, Length: 110, dtype: int64

In [25]:
credit_df[(credit_df['CATEGORIES']=='Transfer,Credit') & (credit_df['merchant_name'].isnull())].shape

(25674, 9)

In [26]:
# Refunds & reversals
search = ['reverse', 'reversal', 'refund', 'return']
credit_df[(credit_df['description'].str.contains('|'.join(search), case=False)) ].shape

(1031, 9)

In [62]:
# POS
search = ['ATM', 'debit card']
pos_df = credit_df[(credit_df['description'].str.contains('|'.join(search), case=False))]

In [63]:
pos_df.shape

(2343, 9)

In [64]:
# POS 2
search = ['POS']
pos_df = credit_df[(credit_df['description'].str.contains('|'.join(search)))]

In [65]:
pos_df.shape

(4688, 9)

In [29]:
pos_df.to_clipboard()

In [30]:
# invoice
search = ['invoice','requested']
idf = credit_df[(credit_df['description'].str.contains('|'.join(search), case=False))]

In [31]:
idf.shape

(309, 9)

In [32]:
idf.to_clipboard()

In [58]:
# recurring
search = ['payroll','recurring','rent']
rec = credit_df[(credit_df['description'].str.contains('|'.join(search), case=False))]

In [59]:
rec.shape

(2280, 9)

In [105]:
df.loc[(df['CATEGORIES'] == 'Transfer,Credit'),'industry_category_name'].value_counts().to_clipboard()

In [48]:
df[(df['categories'] == 'Transfer,Credit') & (df['industry_category_name'] == 'Arts, Entertainment, and Recreation')].sample(4)

,business_id,transaction_id,date,amount,description,merchant_name,type,categories,days_from_current,lending_business_id,Ever15plus,Source,industry_category_name,industry_name,number_of_employees,rev_30,novo_risk_model_score,rs2_bin,rs2_score
60354,dad2647f-ebdb-44b2-8cee-9349b1b6d1a2,k8xmanvMNLsJekr5MqxdszYLaq6b0RHbJLPgE,2023-05-31,-152.40,Card payment,None,credit,"Transfer,Credit",21,5ce6de96-b605-486b-96fc-3a7232a7bb4f,1,FICO,"Arts, Entertainment, and Recreation",Fitness and Recreational Sports Centers,1,1956.7341,277,1,277
59669,33d844ec-b84e-40de-8d83-8dfb3788c0b1,ydOY5BBdvMFV8zx7RV3nsmNY0eQoNKS3qBnMe,2023-01-18,-45.96,WISLOV.COM FL USA Debit Card Purchase,None,credit,"Transfer,Credit",154,1955ba7e-d732-4bd4-9bf4-b55c4398107b,1,FICO,"Arts, Entertainment, and Recreation",All Other Amusement and Recreation Industries,1,1443.5621,708,5,708
32827,2378bec3-b0ba-4ddf-9d4a-a2bbf71cc835,Nao9yNJy9Oud9N4ZPyQPH6Y8ozBMZntpopApN,2023-05-16,-200.00,MSPBNA ACH TRNSFR 230515 79849150906 45237245,Morgan Stanley,credit,"Transfer,Credit",36,28548b41-89b0-452a-840c-7bb6f5b3fdc8,0,GA,"Arts, Entertainment, and Recreation","Independent Artists, Writers, and Performers",1,969.5959,581,3,581
18613,3435a44d-e715-4372-8613-923c64b72d06,Kqeq8QZd3ecM6gZXZbqBuxXNO8XKD5tmZ1Eew,2023-02-22,-101.50,VISA 1270,None,credit,"Transfer,Credit",119,90361232-38ef-49ab-b85f-6c06b20801f3,0,GA,"Arts, Entertainment, and Recreation",All Other Amusement and Recreation Industries,3,4474.5272,746,5,746


In [61]:
df[(df['categories'] == 'Transfer,Deposit')].sample(10)

,business_id,transaction_id,date,amount,description,merchant_name,type,categories,days_from_current,lending_business_id,Ever15plus,Source,industry_category_name,industry_name,number_of_employees,rev_30,novo_risk_model_score,rs2_bin,rs2_score
63241,33b3cd23-d55b-4770-b64b-24fd52c58514,Rz5Eg63M3vfPKVNkya1MhEgmj6zjAVf44YkLO,2023-05-16,-150.00,External Deposit SCHWAB BANK / P2P ID NBR: JES...,None,credit,"Transfer,Deposit",36,d95c9a4d-0ff5-41c7-b92d-a5545baedce0,1,GA,Health Care and Social Assistance,Offices of Physicians (except Mental Health Sp...,1,7934.4982,844,5,844
34755,81aaf0cf-b642-438b-a388-93529129773c,KnyBANNroxFM95KnOZVrcaL5gKDKLeFmL348r,2023-02-28,-4000.00,DEPOSIT / BOHEMIAN LUMBERJ 5FFE9B8A9F,None,credit,"Transfer,Deposit",113,4434d8b0-6d31-4d9d-9ff5-fa27a4609dbd,0,GA,"Agriculture, Forestry, Fishing and Hunting",Logging,1,1963.6042,666,4,666
13204,2993ca66-8214-421c-8f4b-86cbe416b9ea,dbJmZ70pxdTLzN44Mk1KS13p15grYDUoQK65P,2023-03-27,-6000.00,BKOFAMERICA MOBILE 03/26 XXXXX83573 DEPOSIT *M...,None,credit,"Transfer,Deposit",86,feb6a20c-5232-4293-b809-9008a516f917,0,FICO,Real Estate Rental and Leasing,Lessors of Residential Buildings and Dwellings,2,7141.091,659,4,659
56799,37c408e4-a57e-4d98-bd92-e346ae66f628,kPz9AVOQq7h73gExrQOQU47wjXDqDqUZMyJg9N,2023-05-04,-499.31,Deposit Shared Branch Mobile Deposit First Tec...,First Tech Federal Credit Union,credit,"Transfer,Deposit",48,59495dd5-4d03-4783-adf7-b4b4467e92a8,0,FICO,Construction,Residential Remodelers,1,None,None,3,551
52524,829a6684-2f44-465b-a829-7c9de32d3b93,66m94ZmzpktY9kdqkkrXT9zYMxwem9sAP4PQ1,2023-06-01,-1000.00,BKOFAMERICA MOBILE 06/01 XXXXX33992 DEPOSIT *M...,None,credit,"Transfer,Deposit",20,cf50a9fd-da14-4760-b35a-a60a2f918b1f,0,GA,"Arts, Entertainment, and Recreation","Independent Artists, Writers, and Performers",1,1239.9528,698,4,698
3837,48897f91-1c4f-4d4c-8e97-8b463fae9510,VVxwJ9K9mRH5B1pozXzxugrmorwJEncz9vjNO,2023-01-17,-9.00,BANKCARD DES:MTOT DEP ID:XXXXXXXXXX27942 INDN:...,None,credit,"Transfer,Deposit",155,cab150d3-7c94-4a58-9871-4341f1b7276d,0,GA,Other Services,Other Personal Care Services,8,4879.175,887,5,887
29941,7f7b016a-cf50-48f9-9e67-736779fac52e,qV9Ea1q1qNILorQK4gAqixRbNNEMbLfBVXK0y,2023-02-01,-900.00,DEPOSIT/CREDIT,None,credit,"Transfer,Deposit",140,ef1df9cf-8f9b-4526-876c-252c88a55a65,0,FICO,Real Estate Rental and Leasing,Other Activities Related to Real Estate,1,3685.1022,732,5,732
1267,c0c05166-4c30-4199-8721-354a4935ef84,avD51M3MJZT45VknrPPMUR9gpBgrRRI9Manme,2023-04-06,-250.00,MOBILE DEPOSIT,None,credit,"Transfer,Deposit",76,57586ecf-ee91-404c-b562-ca6c0e8b58e8,1,GA,Real Estate Rental and Leasing,Offices of Real Estate Agents and Brokers,2,2810.5514,824,5,824
64142,bbd345ad-3fe9-4a64-b95d-92cea89d1c87,4ag1RjRxJxT8QXzrDx9aUVadbb39xDUqa8wrB,2023-01-10,-175.00,MOBILE DEPOSIT : REF NUMBER :214100353554,None,credit,"Transfer,Deposit",162,189546ea-f958-42c9-86c3-79ed8d255d61,0,GA,Administrative and Support and Waste Managemen...,Janitorial Services,5,2100.3477,810,5,810
59862,df46b691-b5fc-4a81-a93b-75c48275059a,1oZRpZva0VI5LmmeXx4jf5N3gY9KkaIrR4KA1r,2023-02-10,-80.00,POS/Check Card Deposit CASH / APP*CASH OUT Vis...,None,credit,"Transfer,Deposit",131,67a02401-0053-4880-b82a-97939589296e,1,GA,"Arts, Entertainment, and Recreation","Agents and Managers for Artists, Athletes, Ent...",1,1244.6318,404,3,404


In [119]:
df.groupby('BUSINESS_ID')['AMOUNT'].nsmallest(5)

BUSINESS_ID                                
018b2b70-af2d-4f75-be6b-851f1f9d3a1c  3447       -25.00
02fe0ef7-6ddd-4885-aada-e9851fc9f20e  1529    -15050.00
                                      14741   -13917.00
                                      3498    -11605.00
                                      2454     -9067.00
                                                 ...   
ffd8d9b8-4ebb-488b-9167-4d3ea2abe9a3  7794     -5000.00
                                      12670    -1270.99
                                      4451     -1224.36
                                      14187    -1155.00
                                      12083    -1082.72
Name: AMOUNT, Length: 1187, dtype: float64

In [120]:
df[df['BUSINESS_ID']=='02fe0ef7-6ddd-4885-aada-e9851fc9f20e'].sort_values(by='AMOUNT')

,BUSINESS_ID,ACCOUNT_ID,EXTERNAL_ACCOUNT_ID,ID,TRANSACTION_ID,DATE,AMOUNT,PLAID_ACCOUNT_ID,PENDING,DESCRIPTION,...,business_id,Ever15plus,Source,industry_category_name,industry_name,number_of_employees,rev_30,novo_risk_model_score,rs2_bin,rs2_score
1529,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,c9dd00f2-3782-487e-b46e-86ec4e77dab5,nPP9Qm14nYU0pOR44ZMQCJqP4aYzzqHp8NrwN,2023-05-15,-15050.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,EDI PYMNTS Honor Technology Zeewise Inc ACH CR...,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
14741,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,aec90010-515b-4a72-8bfc-ccc7e3a14ea9,bOOb7Z58QYsBpaA113gwi1v96xqwZZhxxea3w,2023-02-01,-13917.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,Payment TGI Friday's Inc 0003Zeewise Inc ACH C...,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
3498,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,8262c255-ab21-4b5f-bb88-fa14b78cde4e,aVVAg78mEYUr84YbbgKoFY1NnOEov6Te88jJw,2023-06-14,-11605.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,Coupa Pay Honor Technology Zeewise Inc ACH CREDIT,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
2454,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,7da675f4-404b-42c1-98c9-68efa8211dee,gnnyQZR8EYIPO5xmmbv6fdvDvRKkzktKkZ4L1,2023-05-09,-9067.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,Payment TGI Friday's Inc 0002Zeewise Inc ACH C...,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
9430,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,670cd978-f119-4d75-a1c9-a0bd47f2c5bf,OkkP389OxYFJedyggqpNtAY5Zgpmg3Tgpw0Mz,2023-02-28,-7010.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,Payment Home Instead Inc ZeewiseInc ACH CREDIT,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
13477,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,452f455d-ca45-4e42-b608-daef5b2b876f,VaarjodQEYc7zqgkkoZEUDw0X8o8jvIakqQoN,2023-01-31,-5825.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,Payment Home Instead Inc ZeewiseInc ACH CREDIT,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
16943,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,4b86c6f9-19e7-49a4-9acc-315c17f58837,E66k9n3by8fKypkLLJwZhdE6NQA8wqtZQXdN9,2022-12-30,-5660.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,Payment Home Instead Inc ZeewiseInc ACH CREDIT,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998
10149,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,1000022135239,484ded0b-e888-4f10-9ac0-bdcfab945c6a,67f830f9-b483-45d6-a849-9ac4fc48fe7b,QvvxKbEJ8YCZg60qqkA9UwJQ8YxmKVswOa9Pn,2022-12-22,-3000.0,kLLNQ6zbEYIpE7oKKvbPfzr3P1V0dqtRvw8Ve,False,WIRE REF# 20221222-00009545 INCOMING WIRE TRAN...,...,02fe0ef7-6ddd-4885-aada-e9851fc9f20e,0,FICO,"Professional, Scientific, and Technical Services",Custom Computer Programming Services,None,None,None,5,998


In [121]:
len(df['BUSINESS_ID'].unique())

250

In [79]:
df[df.index == 52323]

,business_id,transaction_id,date,amount,description,merchant_name,type,categories,days_from_current,lending_business_id,Ever15plus,Source,industry_category_name,industry_name,number_of_employees,rev_30,novo_risk_model_score,rs2_bin,rs2_score
52323,0171a1ff-d47d-4029-b4b2-c1d2b579b857,qMqDDQ0zx9t11MKOB7pbiZ4po3jwyLC110V6y,2023-05-22,-2500.0,Zelle payment from ARLINE CORREA Conf# cmb7ans9u,None,credit,"Transfer,Credit",30,2012588d-a3d2-46c4-8507-46d7e7fd863a,0,GA,Administrative and Support and Waste Managemen...,Travel Agencies,1,1971.8898,620,3,620


In [86]:
df[df['description'].str.contains('greenbox',case=False)]

,business_id,transaction_id,date,amount,description,merchant_name,type,categories,days_from_current,lending_business_id,Ever15plus,Source,industry_category_name,industry_name,number_of_employees,rev_30,novo_risk_model_score,rs2_bin,rs2_score


In [ ]:
loans = ['shopify capital', 'knight', 'expansion', 'payin', 'sba', 'ondeck', 'kabbage', 'bluevine', 'headway', 'prosper', 'onramp','direct capital edi',\
          'lightstream', 'onemain', 'intuit financ', 'ninja', 'good funding', '' ]
lenders = ['dreamspring']